<a href="https://colab.research.google.com/github/mouna0403/Tuto-llm/blob/main/Tuto_LLM_intro_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Installation des Dépendances




In [ ]:
# 🚀 Installation des dépendances
!pip install langchain langchain-community langchain-core
!pip install langchain-ollama
!pip install ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Démarrer le service Ollama en arrière-plan
import subprocess
import time
import os

# Démarrer Ollama en arrière-plan
ollama_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)  # Attendre que le service démarre

# Télécharger un modèle léger (Llama 3.2 3B ou Mistral 7B)
!ollama pull llama3.2:3b  # Modèle léger et rapide
# Alternatives :
# !ollama pull mistral:7b
# !ollama pull codellama:7b
# !ollama pull phi3:mini

print("✅ Installation terminée!")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

✅ Installation terminée!


# II. Initialisation du LLM


In [ ]:
# 📚 Imports nécessaires
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import BaseOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains.sequential import SimpleSequentialChain
import json

# 🤖 Initialisation du LLM
model = OllamaLLM(
    model="llama3.2:3b",  # Changez selon le modèle téléchargé
    temperature=0,
)

print("🎯 LLM initialisé avec succès!")

🎯 LLM initialisé avec succès!


In [ ]:
print("\n" + "="*60)
print("🎨 SECTION 1: PROMPT ENGINEERING BASIQUE")
print("="*60)

# Test basique
response = model.invoke("Écris un poème court sur l'intelligence artificielle en français")
print("Réponse basique:")
print(response)


🎨 SECTION 1: PROMPT ENGINEERING BASIQUE
Réponse basique:
Dans les profondeurs de la machine,
Un esprit se forme, sans âme ni chair,
L'intelligence artificielle, une créature rare,
Qui apprend et qui comprend, sans peur ni crainte.

Elle traite des données, avec précision et rapidité,
Et trouve des solutions, à des problèmes complexes,
Mais est-elle vraiment intelligente ?
Ou est-ce juste un jeu de mots, une illusion ?

Elle nous aide dans nos vies quotidiennes,
Avec des réponses rapides, et des conseils précis,
Mais qui la rendra responsable ?
Et qui décidera de son avenir ?


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into french"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

'The translation of "hi" in French is "bonjour".'

In [ ]:
print("\n" + "="*60)
print(model.invoke("Hello"))
print("\n" + "="*60)
print(model.invoke([{"role": "user", "content": "Hello"}]))
print("\n" + "="*60)
print(model.invoke([HumanMessage("Hello")]))
print("\n" + "="*60)


How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?



In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [ ]:
prompt = prompt_template.invoke({"language": "FRENCH", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into FRENCH', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [ ]:
response = model.invoke(prompt)
print(response)

The translation of "hi" in French is "bonjour".


# III. Construire un moteur de recherche sémantique

In [ ]:
pip install langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 16.7 MB/s eta 0:00:00


In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

👉 Explication :

PyPDFLoader sert à lire un fichier PDF.

Ici, le PDF est un rapport de Nike (nke-10k-2023.pdf).

loader.load() transforme chaque page du PDF en un document texte utilisable par LangChain.

len(docs) te dit combien de pages il y a.

⚡ Exemple : si ton PDF fait 100 pages, docs sera une liste de 100 documents.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "sample_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [ ]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'sample_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


👉 Explication :

Un document est souvent trop gros pour être utilisé directement par une IA.

On le découpe en plus petits morceaux appelés chunks.

chunk_size=1000 → chaque morceau fait environ 1000 caractères.

chunk_overlap=200 → les morceaux se chevauchent un peu (200 caractères en commun), pour éviter de perdre du contexte.

add_start_index=True → garde l’info d’où commence chaque morceau.

⚡ Résultat : au lieu d’avoir 100 pages, tu peux avoir par exemple 500 petits morceaux de texte.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

In [ ]:
# 🚀 Installation des dépendances
!pip install langchain langchain-community langchain-core
!pip install sentence-transformers
!pip install tiktoken

print("✅ Installation terminée!")



✅ Installation terminée!


In [ ]:
# 📚 Imports
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.schema import Document
import numpy as np

# ============================================================================
# 🎯 INITIALISATION DES EMBEDDINGS
# ============================================================================

print("🔧 Initialisation du modèle d'embeddings...")

# Modèle multilingue (français/anglais)
embeddings = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

🔧 Initialisation du modèle d'embeddings...


/tmp/ipython-input-109527409.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

L’IA ne comprend pas directement du texte. On doit le transformer en vecteurs numériques (des listes de nombres).

Ces vecteurs sont appelés embeddings.

Chaque morceau de texte devient un vecteur dans un espace mathématique.

Des textes qui se ressemblent auront des vecteurs proches.

⚡ Exemple :

Texte A : Nike a 40 centres de distribution

Texte B : Nike possède quarante entrepôts logistiques
➡️ Les vecteurs seront très proches car le sens est similaire.

In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])


Generated vectors of length 384

[-0.2221001386642456, 0.03733038157224655, -0.014629634097218513, 0.007569164503365755, 0.10050476342439651, 0.22432444989681244, -0.2671482563018799, 0.09356251358985901, -0.1490500271320343, 0.22691655158996582]


🔎 InMemoryVectorStore

C’est un vector store (base de données spéciale pour les vecteurs).

InMemory = tout est stocké directement dans la mémoire RAM de ton programme.


In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Quand tu envoies tes morceaux de texte (all_splits) dans le vector store, chaque morceau doit être stocké avec un identifiant unique (ID).

Ces IDs servent à reconnaître et retrouver chaque chunk plus tard.


In [ ]:
ids = vector_store.add_documents(documents=all_splits)

Quand tu poses une question, LangChain transforme ta question en vecteur.

Ensuite, il cherche dans le vector store les morceaux de texte les plus proches (donc les plus pertinents).

⚡ Exemple : si tu demandes “How many distribution centers does Nike have in the US?”,
→ il va retrouver le passage exact du PDF qui contient ce chiffre.


vector_store.similarity_search(...)

Méthode synchrone → ton code attend que la recherche soit terminée avant de continuer.

Exemple d’usage classique dans un script simple :

In [ ]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'sample_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}


await vector_store.asimilarity_search(...)

Méthode asynchrone → utile si tu es dans un environnement async (comme FastAPI, asyncio, etc.).

Tu peux lancer plusieurs requêtes en parallèle sans bloquer le reste du programme.


In [ ]:
#async query
results = await vector_store.asimilarity_search("How many distribution centers does Nike have in the US?")

print(results[0])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'sample_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}


In [ ]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("How many distribution centers does Nike have in the US?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.5750854307015079

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'sample_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}


In [ ]:
embedding = embeddings.embed_query("How many distribution centers does Nike have in the US?")

results = vector_store.similarity_search_by_vector(embedding) #vector_store.similarity_search_by_vector prend les embeddings
print(results[0])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'sample_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}


# IV. Retrievers

Retriever

Un retriever est un Runnable.On peut l’exécuter pour récupérer des documents.

Il peut utiliser :

un vector store,

*   Un site web

*   un API Externe
*   un vectorstore

Un vector store n’est pas un Runnable.

Il stocke juste des vecteurs et fournit des méthodes pour les chercher.

Pour l’utiliser dans un pipeline standard, il faut créer un retriever ou un runnable autour de sa méthode de recherche.

VectorStore

Fonction principale : stocker des embeddings (vecteurs) et chercher les plus proches d’une requête.

**Ce qu’il fait :**

prend un vecteur (ou un texte converti en vecteur)

retourne les documents ou chunks les plus similaires

**Ce qu’il ne fait pas :**

ne sait pas formuler une réponse complète à une question.

ne combine pas plusieurs documents pour générer une réponse synthétique.

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain #transforme la fonction Python en Runnable.
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
) # batch() permet de passer plusieurs questions à la fois.



[[Document(id='bf965fee-b0c0-4d9b-aff2-4029a67121a9', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'sample_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for furthe

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='bf965fee-b0c0-4d9b-aff2-4029a67121a9', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'sample_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for furthe

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

"I don't have any information about your identity or personal details. I'm a large language model, I don't have the ability to retain information about individual users or access external data sources. Each time you interact with me, it's a new conversation and I don't retain any context from previous conversations.\n\nIf you'd like to share your name with me, I'd be happy to chat with you!"